#### [Return to README ](https://github.com/teresahanak/wikipedia-life-expectancy#explore-the-project)

# Wikipedia Notable Life Expectancies
## Notebook  14: Model User Interface
### Contents

This notebook covers:
- An example Gradio user interface for using or testing the productionized model.

### Instructions
Options for Running this Notebook:  
1. Clone the repository:   
    - Navigate to the cloned repository in Jupyter and open the notebook to run it.
    - Refer to [Application and Package Versions](https://github.com/teresahanak/wikipedia-life-expectancy#application-and-package-versions).
2. Save the notebook and files below to the same working directory:<sup>1</sup>
    - Download the notebook:
        - Click on <kbd>Raw</kbd> above.
        - Save <kbd>Ctrl</kbd> + <kbd>S</kbd>
    - Follow the link to the pkl file and click <kbd>Download</kbd>, then move file to the same directory as the notebook:
        - [production_model.pkl]()
    - In Jupyter, navigate to the directory in which the notebook and file are saved, to open and run the notebook.  
    - Refer to [Application and Package Versions](https://github.com/teresahanak/wikipedia-life-expectancy#application-and-package-versions).

    

### Production Model Features Dictionary
- **num_references**:  Number of references for individual's page
- **years**: Translation of year pf death($year - 1994$)
- **sciences**: (0 for no or 1 for yes) individual known for sciences (math, physics, chemistry, engineering, mechanics, etc.)
- **social**: (0 for no or 1 for yes) individual known for social action (philanthropy, fund-raising for social cause, founder of charity, etc.)
- **spiritual**: (0 for no or 1 for yes) individual known for spiritual association (religious association, traditional healing, self-help/motivational instructor, etc.)
- **academia_humanities**: (0 for no or 1 for yes) individual known for education activity (educator, education administration, lecturer, etc.; excludes sports-related instruction/coaching, but includes art/performing arts instruction; includes museum-related activities; classics, archeology, linguistics, anthropology, etc.)
- **business_farming**: (0 for no or 1 for yes) individual known for business or farming (includes marketing, millionaire/billionaire, manufacturing, oil/energy)
- **arts**: (0 for no or 1 for yes) individual known for arts-related activity (fine and performing arts, journalism, writing, arts administration, art patronage, collecting, etc.; gallery owners/founders are included; museum-related is excluded; stunt performers included)
- **sports**: (0 for no or 1 for yes) individual known for sports-related activity (traditional sports participation/instruction/coaching/ownership/fandom/commentator and anything competition-based, including non-physical games, such as chess; sportswriter, etc. would have dual category of arts)
- **law_enf_military_operator**: (0 for no or 1 for yes) individual known for law enforcement, military/paramilitary association/activity, or specialized equipment operation (pilot, ship captain (non-sport), radio operator, etc.); category aims to reflect individual's proximity to activity and/or weapons/equipment or decision-making that could impact life span, independent of legality of activity
- **politics_govt_law**: (0 for no or 1 for yes) individual known for political activity (official or activism), participation in legal system (lawyer, judge, etc.), nobility or inherited status; directly or by marriage; union activity is included
- **crime**: (0 for no or 1 for yes) individual known for criminal activity; category aims for "innocent until proven guilty"; includes convicted criminals (can be for a different crime); includes individuals labeled "terrorist", but generally excludes individuals awaiting trial
- **num_categories**: Total number of known for categories for individual
- **region_**: One hot encoded (0 for no or 1 for yes) ultimate geographical region of residency as follows:
    - **region_Asia**
    - **region_Central Asia**
    - **region_Europe**
    - **region_Mid-Cent America/Caribbean**
    - **region_Middle East**
    - **region_North America**
    - **region_Oceania**
    - **region_Russian Federation**
    - **region_South America**
    - **region_South East Asia**
- **prior_region_**: One hot encoded (0 for no or 1 for yes) prior geographical region of residency, with option of "No Prior Region", as follows:
    - **prior_region_Asia**
    - **prior_region_Central Asia**
    - **prior_region_Europe**
    - **prior_region_Mid-Cent America/Caribbean**
    - **prior_region_Middle East**
    - **prior_region_No Prior Region**
    - **prior_region_North America**
    - **prior_region_Oceania**
    - **prior_region_Russian Federation**
    - **prior_region_South America**
    - **prior_region_South East Asia**

### Importing Libraries

In [20]:
# To structure code automatically
%load_ext nb_black

# To import/export sqlite databases
# import sqlite3 as sql

# To save/open python objects in pickle file
import pickle

# To help with reading, cleaning, and manipulating data
import pandas as pd
import numpy as np

# To create user interface for predictions
import gradio as gr
from gradio.components import Slider

# To define maximum number of columns to be displayed in a dataframe
# pd.set_option("display.max_columns", None)
# To define the maximum number of rows to be displayed in a dataframe
# pd.set_option("display.max_rows", 200)

# To supress warnings
# import warnings

# warnings.filterwarnings("ignore")

# To set some visualization attributes
# pd.set_option("max_colwidth", 150)

# To play auditory cue when cell has executed, has warning, or has error and set chime theme
import chime

chime.theme("zelda")

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

### Loading Productionized Model

In [3]:
# Loading Productionized Model
with open("production_model.pkl", "rb") as f:
    production_model = pickle.load(f)

<IPython.core.display.Javascript object>

### Loading `region_place_dict`

In [67]:
# Loading region_place_dict
with open("region_place_dict.pkl", "rb") as f:
    region_place_dict = pickle.load(f)

# Checking region_place_dict
region_place_dict

{'North America': ['Canada', 'United States of America', 'North America'],
 'Mid-Cent America/Caribbean': ['Martinique',
  'Saint Vincent and the Grenadines',
  'Antigua and Barbuda',
  'Saint Kitts and Nevis',
  'Saint Lucia',
  'Grenada',
  'Belize',
  'The Bahamas',
  'El Salvador',
  'Nicaragua',
  'Panama',
  'Costa Rica',
  'Barbados',
  'Guatemala',
  'Haiti',
  'Honduras',
  'Central America and the Caribbean',
  'Dominican Republic',
  'Trinidad and Tobago',
  'Puerto Rico',
  'Cuba',
  'Jamaica',
  'Mexico'],
 'South America': ['Suriname',
  'Paraguay',
  'Bolivia',
  'Guyana',
  'Ecuador',
  'Peru',
  'Uruguay',
  'Venezuela',
  'Colombia',
  'Chile',
  'Argentina',
  'Brazil'],
 'Europe': ['Vatican City State',
  'Gibraltar',
  'Liechtenstein',
  'San Marino',
  'Andorra',
  'Monaco',
  'Luxembourg',
  'Montenegro',
  'Moldova',
  'Europe',
  'Macedonia, Republic of',
  'Malta',
  'Latvia',
  'Iceland',
  'Belarus',
  'Slovakia',
  'Bosnia and Herzegovina',
  'Albania',
  '

<IPython.core.display.Javascript object>

In [87]:
# Creating places list
places = sorted([place for region in region_place_dict.values() for place in region])

# Creating known_for categories list
known_for = [
    "sciences",
    "social",
    "spiritual",
    "academia_humanities",
    "business_farming",
    "arts",
    "sports",
    "law_enf_military_operator",
    "politics_govt_law",
    "crime",
]

<IPython.core.display.Javascript object>

In [88]:
def predict_age(
    num_references,
    sciences,
    social,
    spiritual,
    academia_humanities,
    business_farming,
    arts,
    sports,
    law_enf_military_operator,
    politics_govt_law,
    crime,
    num_categories,
    years,
    region,
    prior_region,
):
    X = pd.DataFrame(
        {
            "num_references": num_references,
            "years": (year - 1994),
            "sciences": sciences,
            "social": social,
            "spiritual": spiritual,
            "academia_humanities": academia_humanities,
            "business_farming": business_farming,
            "arts": arts,
            "sports": sports,
            "law_enf_military_operator": law_enf_military_operator,
            "politics_govt_law": politics_govt_law,
            "crime": crime,
            "num_categories": num_categories,
            "region_Asia": region_Asia,
            "region_Central Asia": region_Central_Asia,
            "region_Europe": region_Europe,
            "region_Mid-Cent America/Caribbean": region_Mid_Cent_Amer_Caribb,
            "region_Middle East": region_Middle_East,
            "region_North America": region_North_America,
            "region_Oceania": region_Oceania,
            "region_Russian Federation": region_Russian_Federation,
            "region_South America": region_South_America,
            "region_South East Asia": region_SE_Asia,
            "prior_region_Asia": prior_region_Asia,
            "prior_region_Central Asia": prior_region_Central_Asia,
            "prior_region_Europe": prior_region_Europe,
            "prior_region_Mid-Cent America/Caribbean": prior_region_Mid_Cent_Amer_Caribb,
            "prior_region_Middle East": prior_region_Middle_East,
            "prior_region_No Prior Region": prior_region_None,
            "prior_region_North America": prior_region_North_America,
            "prior_region_Oceania": prior_region_Oceania,
            "prior_region_Russian Federation": prior_region_Russian_Federation,
            "prior_region_South America": prior_region_South_America,
            "prior_region_South East Asia": prior_region_SE_Asia,
        }
    )
    return X

<IPython.core.display.Javascript object>

In [136]:
def summary(year, num_references, prior_region, region, known_for_boxes):
    return f"You are predicting the life span for a Wikipedia Notable, who died in {year}, with {num_references} references, originally from {prior_region}, and who ultimately resided in {region}.  The individual was known for: {known_for_boxes}."

<IPython.core.display.Javascript object>

In [138]:
with gr.Blocks() as demo:
    gr.Markdown("Make age predictions for Wikipedia Notables using production model.")
    year = Slider(1994, 2022, label="Year of Notable's Death", interactive=True, step=1)
    num_references = Slider(
        0,
        700,
        label="Number of references on Notable's individual Wikipedia page",
        interactive=True,
        step=1,
    )
    region = gr.Dropdown(
        choices=places, label="Notable's ultimate geographical location of residency"
    )
    prior_region = gr.Dropdown(
        choices=places, label="Notable's geographical location of residency at birth"
    )
    known_for_boxes = gr.CheckboxGroup(
        choices=known_for,
        label="Category or categories for which the Notable was known (see category definitions in model features dictionary; Example: sports broadcaster would have categories of sports and arts)",
    )
    output = gr.Textbox(label="Summary of Input")
    predict_btn = gr.Button("Predict Notable's Age")
    predict_btn.click(
        fn=summary,
        inputs=[year, num_references, region, prior_region, known_for_boxes],
        outputs=output,
    )


demo.launch()

Running on local URL:  http://127.0.0.1:7918

To create a public link, set `share=True` in `launch()`.


(<gradio.routes.App at 0x20af4fd5f70>, 'http://127.0.0.1:7918/', None)

<IPython.core.display.Javascript object>

In [ ]:
print('Complete')

# Chime notification when cell executes
chime.success()

#### [Return to README ](https://github.com/teresahanak/wikipedia-life-expectancy#explore-the-project)